In [1]:
import pandas as pd
import numpy as np
import copy
import sympy

np.random.seed(42)

In [2]:
def generate_instance():
    # iç veri
    ay = np.max([1.0, np.round(np.random.normal(3, 0.5))]) if np.random.rand() > 0.95 else np.max([1.0, np.round(np.random.normal(28, 10))])
    kredi_karti_baska_banka = 1.0 if np.random.rand() > 0.9 else 0.0
    kredi_karti_aldi = 1.0 if np.random.rand() > 0.99 else 0.0
    kredi_karti_banka = 1.0 if np.random.rand() > 0.7 else 0.0
    
    kredi_karti_ilk_ay_harcama = 0.0
    
    if kredi_karti_banka == 1.0 or kredi_karti_aldi == 1.0:
        kredi_karti_ilk_ay_harcama = np.max([0.0, np.random.normal(1000.0, 250.0)])
    
    kredi_karti_harcama = 0.0
    if kredi_karti_banka == 1.0 or kredi_karti_aldi == 1.0 or kredi_karti_baska_banka == 1.0:
        kredi_karti_harcama = np.max([0.0, np.random.normal(-500, 1000)]) if np.random.rand() > 0.4 else np.max([0.0, np.random.normal(3000, 2000)])
    
    kredi_karti_harcama_onceki_ay = 0.0
    if ay > 1 and (kredi_karti_banka == 1.0 or kredi_karti_aldi == 1.0 or kredi_karti_baska_banka == 1.0):
        eps = np.random.normal(100, 10)
        kredi_karti_harcama_onceki_ay = np.max([0.0, np.random.normal(500, 500) + eps]) if np.random.rand() > 0.4 else np.max([0.0, np.random.normal(3000, 2000) + eps])
    
    toplam_varlik = np.max([0.0, np.random.normal(-1000, 4000)]) if np.random.rand() > 0.4 else np.max([0.0, np.random.normal(10000, 8000)])
    toplam_varlik_onceki_ay = 0.0
    toplam_varlik_iki_onceki_ay = 0.0
    toplam_varlik_onceki_3_ay_ortalama = 0.0
    if ay > 3:
        eps = np.random.normal(200, 25)
        toplam_varlik_onceki_ay = np.max([0.0, np.random.normal(2500, 3000) + eps]) if np.random.rand() > 0.5 else np.max([0.0, np.random.normal(10000, 8000) + eps])
        eps = np.random.normal(200, 25)
        toplam_varlik_iki_onceki_ay = np.max([0.0, np.random.normal(2500, 3000) + eps]) if np.random.rand() > 0.5 else np.max([0.0, np.random.normal(10000, 8000) + eps])
        eps = np.random.normal(200, 25)
        toplam_varlik_uc_onceki_ay = np.max([0.0, np.random.normal(2500, 3000) + eps]) if np.random.rand() > 0.5 else np.max([0.0, np.random.normal(10000, 8000) + eps])
        toplam_varlik_onceki_3_ay_ortalama = (toplam_varlik_onceki_ay + toplam_varlik_iki_onceki_ay + toplam_varlik_uc_onceki_ay) / 3
    
    acik_kredi = 1.0 if np.random.rand() > 0.83 else 0.0
    onceki_ay_urun_sayisi = np.max([0.0, np.round(np.random.normal(4.0, 1.5))])
    urun_sayisi = 0.0 if np.random.rand() > 0.9 else np.max([0.0, np.round(np.random.normal(3.0, 2.0))])
    
    # performans ölçütleri
    yeni_musteri = 1.0 if ay <= 3.0 else 0.0
    capraz_satis = 1.0 if ay > 3.0 and (kredi_karti_baska_banka == 0.0 and kredi_karti_ilk_ay_harcama > 100.0) and np.random.rand() > 0.8 else 0.0
    yukari_satis = 1.0 if ay > 3.0 and (toplam_varlik_onceki_ay > 10000 and toplam_varlik / toplam_varlik_onceki_3_ay_ortalama > 1.5) else 0.0
    
    devam_eden_musteri = 0.0
    terk = 0.0
    geri_kazanilmis_musteri = 0.0
    pasif_musteri = 0.0
    aktiflestirilmis_musteri = 0.0
    
    if ay > 3 and kredi_karti_harcama == 0.0 and toplam_varlik < 50.0 and acik_kredi == 0 and urun_sayisi == 0:
        pasif_musteri = 1.0
    elif ay > 3 and (onceki_ay_urun_sayisi > 0 or toplam_varlik_onceki_ay > 0 or kredi_karti_harcama_onceki_ay > 0) and urun_sayisi == 0 and toplam_varlik < 50:
        terk = 1.0
    elif ay > 3 and toplam_varlik_onceki_ay == 0.0 and kredi_karti_harcama_onceki_ay == 0.0 and toplam_varlik_onceki_ay == 0.0 and urun_sayisi > 0 and toplam_varlik > 0:
        aktiflestirilmis_musteri = 1.0
    elif ay > 3 and onceki_ay_urun_sayisi == 0.0 and urun_sayisi > 0:
        geri_kazanilmis_musteri = 1.0
    elif ay > 3:
        devam_eden_musteri = 1.0
    
    #if tavsiye > 0.7 and np.random.rand() > 0.9:
    #    capraz_satis = 1.0
    #if tavsiye > 0.8 and memnuniyet > 0.8 and np.random.rand() > 0.9:
    #    yukaris_satis = 1.0
    
    
    # digital
    basarili_giris_sayisi = 0.0
    gezdigi_sayfa_sayisi = 0.0
    hata_islem_sayisi = 0.0
    islem_ortalama_tiklama = 0.0
    cevrim_ici_sure = 0.0
    
    # classic
    personel_ilgisi = 0.0
    atm_kullanim = 0.0
    sube_kullanim = 0.0
    sube_sure = 0.0
    
    # social media
    takip = 0.0
    begeni = 0.0
    pozitif_yorum = np.max([0.0, np.round(np.random.normal(0.7, 0.75))])
    negatif_yorum = np.max([0.0, np.round(np.random.normal(0.3, 0.15))])
    
    gelir = 0.0
    
    if yeni_musteri == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(5, 0.5))])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(2, 0.35))])
        hata_islem_sayisi = np.max([0.0, np.round(np.random.normal(0.9, 0.4))])
        islem_ortalama_tiklama = np.max([0.0, np.round(np.random.normal(6, 0.5))])
        cevrim_ici_sure = np.max([0.0, np.random.normal(6, 0.5)])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(1, 0.1))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(1, 0.2))])
        sube_sure = np.max([0.0, np.random.normal(0.5, 0.3)])
        personel_ilgisi = np.max([0.0, np.random.normal(3.0, 0.15)]) if sube_kullanim > 0 else 3.5
        takip = 1.0 if np.random.rand() > 0.7 else 0.0
        begeni = 1.0 if np.random.rand() > 0.95 else 0.0
        pozitif_yorum = np.max([0.0, np.round(np.random.normal(0.7, 0.75))])
        negatif_yorum = np.max([0.0, np.round(np.random.normal(0.3, 0.15))])
        gelir = np.max([0.0, np.round(np.random.normal(500, 100))])
    elif devam_eden_musteri == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(15, 0.25))])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(4, 0.5))])
        hata_islem_sayisi = np.max([0.0, np.round(np.random.normal(1.1, 0.2))])
        islem_ortalama_tiklama = np.max([0.0, np.round(np.random.normal(4, 0.5))])
        cevrim_ici_sure = np.max([0.0, np.random.normal(10, 0.5)])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(3, 0.25))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(2, 0.2))])
        sube_sure = np.max([0.0, np.random.normal(1.0, 0.3)])
        personel_ilgisi = np.max([0.0, np.random.normal(3.5, 0.15)]) if sube_kullanim > 0 else 3.5
        takip = 1.0 if np.random.rand() > 0.5 else 0.0
        begeni = 1.0 if np.random.rand() > 0.85 else 0.0
        pozitif_yorum = np.max([0.0, np.round(np.random.normal(1.1, 0.85))])
        negatif_yorum = np.max([0.0, np.round(np.random.normal(0.4, 0.22))])
        gelir = np.max([0.0, np.round(np.random.normal(8000, 4000))])
    elif terk == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(1.5, 0.2))])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(2.0, 0.75))])
        hata_islem_sayisi = np.max([0.0, np.round(np.random.normal(4.0, 0.5))])
        islem_ortalama_tiklama = np.max([0.0, np.round(np.random.normal(9, 0.8))])
        cevrim_ici_sure = np.max([0.0, np.random.normal(2, 0.5)])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(0.5, 0.05))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(3, 0.5))])
        sube_sure = np.max([0.0, np.random.normal(6.0, 0.5)])
        personel_ilgisi = np.max([0.0, np.random.normal(1.5, 0.19)]) if sube_kullanim > 0 else 3.5
        takip = 1.0 if np.random.rand() > 0.6 else 0.0
        begeni = 1.0 if np.random.rand() > 0.995 else 0.0
        pozitif_yorum = np.max([0.0, np.round(np.random.normal(0.3, 0.25))])
        negatif_yorum = np.max([0.0, np.round(np.random.normal(2.8, 0.5))])
        gelir = np.max([0.0, np.round(np.random.normal(100, 50))])
    elif geri_kazanilmis_musteri == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(3, 0.1))])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(1.5, 0.3))])
        hata_islem_sayisi = np.max([0.0, np.round(np.random.normal(0.2, 0.1))])
        islem_ortalama_tiklama = np.max([0.0, np.round(np.random.normal(6, 0.5))])
        cevrim_ici_sure = np.max([0.0, np.random.normal(4, 0.2)])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(0.75, 0.2))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(2.5, 0.1))])
        sube_sure = np.max([0.0, np.random.normal(0.25, 0.3)])
        personel_ilgisi = np.max([0.0, np.random.normal(4.5, 0.35)]) if sube_kullanim > 0 else 3.5
        takip = 1.0 if np.random.rand() > 0.6 else 0.0
        begeni = 1.0 if np.random.rand() > 0.96 else 0.0
        pozitif_yorum = np.max([0.0, np.round(np.random.normal(0.85, 0.17))])
        negatif_yorum = np.max([0.0, np.round(np.random.normal(0.4, 0.42))])
        gelir = np.max([0.0, np.round(np.random.normal(1000, 600))])
    elif aktiflestirilmis_musteri == 1.0:
        basarili_giris_sayisi = np.max([0.0, np.round(np.random.normal(4.3, 0.2))])
        gezdigi_sayfa_sayisi = np.max([0.0, np.round(np.random.normal(2, 0.5))])
        hata_islem_sayisi = np.max([0.0, np.round(np.random.normal(0.1, 0.1))])
        islem_ortalama_tiklama = np.max([0.0, np.round(np.random.normal(4, 0.5))])
        cevrim_ici_sure = np.max([0.0, np.random.normal(5, 0.5)])
        atm_kullanim = np.max([0.0, np.round(np.random.normal(1.2, 0.4))])
        sube_kullanim = np.max([0.0, np.round(np.random.normal(0.5, 0.1))])
        personel_ilgisi = np.max([0.0, np.random.normal(3.95, 0.45)]) if sube_kullanim > 0 else 3.5
        sube_sure = np.max([0.0, np.random.normal(0.25, 0.2)])
        takip = 1.0 if np.random.rand() > 0.76 else 0.0
        begeni = 1.0 if np.random.rand() > 0.99 else 0.0
        pozitif_yorum = np.max([0.0, np.round(np.random.normal(1.32, 0.39))])
        negatif_yorum = np.max([0.0, np.round(np.random.normal(0.5, 0.35))])
        gelir = np.max([0.0, np.round(np.random.normal(2500, 900))])
        
    tavsiye = np.random.normal(6.5, 0.25) # bias
    tavsiye += yeni_musteri * np.random.normal(1.5, 0.2)
    tavsiye += capraz_satis * np.random.normal(2, 0.5)
    tavsiye += yukari_satis * np.random.normal(4, 0.1)
    tavsiye += devam_eden_musteri * np.random.normal(0.5, 0.05)
    tavsiye += terk * np.random.normal(-4.0, 0.002)
    tavsiye += geri_kazanilmis_musteri * np.random.normal(1.0, 0.5)
    tavsiye += aktiflestirilmis_musteri * np.random.normal(0.5, 0.75)
    tavsiye += pasif_musteri * np.random.normal(-1.75, 0.55)
    
    tavsiye += basarili_giris_sayisi * np.random.normal(0.25, 0.2)
    tavsiye += gezdigi_sayfa_sayisi * np.random.normal(0.11, 0.05)
    tavsiye += hata_islem_sayisi * np.random.normal(-2.5, 0.01)
    tavsiye += (islem_ortalama_tiklama - 5.15) * np.random.normal(-0.3, 0.35)
    tavsiye += (cevrim_ici_sure - np.random.normal(4.15, 0.35)) * np.random.normal(0.3, 0.1)
    
    tavsiye += (personel_ilgisi - 3.5) * np.random.normal(0.7, 0.2)
    tavsiye += atm_kullanim * np.random.normal(0.5, 0.4)
    tavsiye += sube_kullanim * np.random.normal(0.1, 0.05)
    tavsiye += (sube_sure - 0.25) * np.random.normal(-2.5, 0.05)
    
    tavsiye += begeni * np.random.normal(0.5, 0.25)
    tavsiye += takip * np.random.normal(0.1, 0.01)
    tavsiye += pozitif_yorum * np.random.normal(0.9, 0.4)
    tavsiye += negatif_yorum * np.random.normal(-1.8, 0.1)
    
    tavsiye += (urun_sayisi - 3.75) * np.random.normal(0.85, 0.3)
    tavsiye += 0 if toplam_varlik > 50 else np.random.normal(-0.75, 0.28)
    tavsiye += (acik_kredi - 1) * np.random.normal(0.3, 0.07)
    tavsiye += np.random.normal(-0.75, 0.15) if (onceki_ay_urun_sayisi - urun_sayisi) and urun_sayisi == 0 > 0 else 0.0

    tavsiye += np.random.normal(-0.5, 0.15)
    tavsiye = np.round(tavsiye)
    tavsiye = np.max([0.0, tavsiye])
    tavsiye = np.min([10.0, tavsiye])
    
    memnuniyet = np.random.normal(30.0, 4.0)
    
    memnuniyet += yeni_musteri * np.random.normal(1.5, 0.5)
    memnuniyet += capraz_satis * np.random.normal(14, 2.0)
    memnuniyet += yukari_satis * np.random.normal(35, 5.0)
    memnuniyet += devam_eden_musteri * np.random.normal(3.0, 1.0)
    memnuniyet += terk * np.random.normal(-35.0, 5.0)
    memnuniyet += geri_kazanilmis_musteri * np.random.normal(1.25, 2.5)
    memnuniyet += aktiflestirilmis_musteri * np.random.normal(1.5, 0.95)
    memnuniyet += pasif_musteri * np.random.normal(-15.5, 3.75)
    
    memnuniyet += basarili_giris_sayisi * np.random.normal(3, 1.1)
    memnuniyet += ay * np.random.normal(1.5, 0.2)
    memnuniyet += gezdigi_sayfa_sayisi * np.random.normal(0.65, 0.15)
    memnuniyet += hata_islem_sayisi * np.random.normal(-8.5, 1.65)
    memnuniyet += (islem_ortalama_tiklama - 5.5) * np.random.normal(-1.2, 0.25)
    memnuniyet += (cevrim_ici_sure - np.random.normal(4.5, 0.5)) * np.random.normal(0.9, 0.3)
    
    memnuniyet += (personel_ilgisi - 3.5) * np.random.normal(5.7, 1.2)
    memnuniyet += atm_kullanim * np.random.normal(3.2, 0.62)
    memnuniyet += sube_kullanim * np.random.normal(-0.3, 0.5)
    memnuniyet += (sube_sure - 0.25) * np.random.normal(-10.5, 0.5)
    
    memnuniyet += begeni * np.random.normal(0.5, 0.15)
    memnuniyet += takip * np.random.normal(0.1, 0.01)
    memnuniyet += pozitif_yorum * np.random.normal(1.6, 0.32)
    memnuniyet += negatif_yorum * np.random.normal(-8.2, 1.3)
    
    memnuniyet += (urun_sayisi - 3) * np.random.normal(1.0, 0.01)
    memnuniyet += np.random.normal(1.0, 0.25) if toplam_varlik > 50 else np.random.normal(-5.25, 2.6)
    memnuniyet += (kredi_karti_banka - 1) * np.random.normal(7.0, 1.5)
    memnuniyet += np.random.normal(-6.5, 2.35) if (onceki_ay_urun_sayisi - urun_sayisi) and urun_sayisi == 0 > 0 else 0.0
    
    memnuniyet += np.random.normal(-12, 3.0)
    memnuniyet = np.round(memnuniyet)
    memnuniyet = np.max([0.0, memnuniyet])
    memnuniyet = np.min([100.0, memnuniyet])
    
    instance = [tavsiye, memnuniyet,
                #ay,
                #kredi_karti_baska_banka, kredi_karti_aldi, kredi_karti_banka, kredi_karti_ilk_ay_harcama, kredi_karti_harcama, kredi_karti_harcama_onceki_ay,
                #toplam_varlik, toplam_varlik_onceki_ay, toplam_varlik_iki_onceki_ay, toplam_varlik_onceki_3_ay_ortalama,
                #acik_kredi,
                #onceki_ay_urun_sayisi, urun_sayisi,
                yeni_musteri, capraz_satis, yukari_satis, devam_eden_musteri, terk, geri_kazanilmis_musteri, pasif_musteri, aktiflestirilmis_musteri,
                basarili_giris_sayisi, gezdigi_sayfa_sayisi, hata_islem_sayisi, islem_ortalama_tiklama, cevrim_ici_sure,
                sube_kullanim, sube_sure, atm_kullanim, personel_ilgisi,
                takip, begeni, pozitif_yorum, negatif_yorum,
                gelir
               ]
    
    return np.array(instance)
    
    
    
    

In [3]:
samples = np.array([generate_instance() for x in range(20000)])
columns = ["tavsiye", "memnuniyet",
                #"ay",
                #"kredi_karti_baska_banka", "kredi_karti_aldi", "kredi_karti_banka", "kredi_karti_ilk_ay_harcama", "kredi_karti_harcama", "kredi_karti_harcama_onceki_ay",
                #"toplam_varlik", "toplam_varlik_onceki_ay", "toplam_varlik_iki_onceki_ay", "toplam_varlik_onceki_3_ay_ortalama",
                #"acik_kredi",
                #"onceki_ay_urun_sayisi", "urun_sayisi",
                "yeni_musteri", "capraz_satis", "yukari_satis", "devam_eden_musteri", "terk", "geri_kazanilmis_musteri", "pasif_musteri", "aktiflestirilmis_musteri",
                "basarili_giris_sayisi", "gezdigi_sayfa_sayisi", "hata_islem_sayisi", "islem_ortalama_tiklama", "cevrim_ici_sure",
                "sube_kullanim", "sube_sure","atm_kullanim", "personel_ilgisi",
                "takip", "begeni", "pozitif_yorum", "negatif_yorum",
          "gelir"]
data = pd.DataFrame(samples, columns = columns)

In [4]:
data["yeni_musteri"].sum() / data.shape[0]

0.04905

In [5]:
data["capraz_satis"].sum() / data.shape[0]

0.0518

In [6]:
data["yukari_satis"].sum() / data.shape[0]

0.0312

In [7]:
data["devam_eden_musteri"].sum() / data.shape[0]

0.827

In [8]:
data["terk"].sum() / data.shape[0]

0.0253

In [9]:
data["geri_kazanilmis_musteri"].sum() / data.shape[0]

0.0071

In [10]:
data["pasif_musteri"].sum() / data.shape[0]

0.04705

In [11]:
#data["aktiflestirilmis_musteri"].sum() / data.shape[0]

In [12]:
data["tavsiye"].mean()

7.49525

In [13]:
data["memnuniyet"].mean()

81.8984

In [14]:
data.var()

tavsiye                     9.936574e+00
memnuniyet                  7.847917e+02
yeni_musteri                4.664643e-02
capraz_satis                4.911922e-02
yukari_satis                3.022807e-02
devam_eden_musteri          1.430782e-01
terk                        2.466114e-02
geri_kazanilmis_musteri     7.049942e-03
pasif_musteri               4.483854e-02
aktiflestirilmis_musteri    4.252188e-02
basarili_giris_sayisi       2.201259e+01
gezdigi_sayfa_sayisi        1.381454e+00
hata_islem_sayisi           3.603491e-01
islem_ortalama_tiklama      1.922466e+00
cevrim_ici_sure             7.434273e+00
sube_kullanim               3.679340e-01
sube_sure                   8.137380e-01
atm_kullanim                8.518080e-01
personel_ilgisi             6.766195e-01
takip                       2.481375e-01
begeni                      1.078764e-01
pozitif_yorum               6.817413e-01
negatif_yorum               3.771069e-01
gelir                       2.034140e+07
dtype: float64

In [15]:
data = (data - data.min()) / (data.max() - data.min())

In [16]:
data.cov()

,tavsiye,memnuniyet,yeni_musteri,capraz_satis,yukari_satis,devam_eden_musteri,terk,geri_kazanilmis_musteri,pasif_musteri,aktiflestirilmis_musteri,...,cevrim_ici_sure,sube_kullanim,sube_sure,atm_kullanim,personel_ilgisi,takip,begeni,pozitif_yorum,negatif_yorum,gelir
tavsiye,0.099366,0.050084,-0.009145,0.004200,0.006615,0.055846,-0.018964,0.000393,-0.032177,0.004046,...,0.041118,0.012084,-0.009397,0.038907,0.028815,0.019188,0.010394,0.016249,-0.011512,0.018920
memnuniyet,0.050084,0.078479,-0.028357,0.004740,0.005414,0.083194,-0.020721,-0.000869,-0.029821,-0.003425,...,0.050283,0.017945,-0.007985,0.051751,0.029103,0.024442,0.011047,0.011011,-0.009310,0.026918
yeni_musteri,-0.009145,-0.028357,0.046646,-0.002541,-0.001530,-0.040566,-0.001241,-0.000348,-0.002308,-0.002183,...,-0.011837,-0.010060,-0.003321,-0.019647,-0.002467,-0.007652,-0.003683,-0.003938,-0.002927,-0.012930
capraz_satis,0.004200,0.004740,-0.002541,0.049119,0.000634,0.004012,0.001140,0.000082,-0.000787,-0.001905,...,0.001501,0.002040,0.001190,0.001940,0.000223,0.001443,0.000679,0.000223,0.000434,0.000822
yukari_satis,0.006615,0.005414,-0.001530,0.000634,0.030228,0.005198,-0.000789,-0.000022,-0.001468,-0.001388,...,0.002852,0.001409,-0.000041,0.002978,0.001306,0.000851,0.000662,0.000557,-0.000361,0.001329
devam_eden_musteri,0.055846,0.083194,-0.040566,0.004012,0.005198,0.143078,-0.020924,-0.005872,-0.038912,-0.036803,...,0.078419,0.037489,-0.001916,0.082222,0.033704,0.040511,0.017980,0.010745,-0.007974,0.043176
terk,-0.018964,-0.020721,-0.001241,0.001140,-0.000789,-0.020924,0.024661,-0.000180,-0.001190,-0.001126,...,-0.014591,0.007630,0.016635,-0.013322,-0.008196,-0.002005,-0.002912,-0.005321,0.012478,-0.007097
geri_kazanilmis_musteri,0.000393,-0.000869,-0.000348,0.000082,-0.000022,-0.005872,-0.000180,0.007050,-0.000334,-0.000316,...,-0.002891,0.001208,-0.000704,-0.003069,0.001553,-0.000093,-0.000473,-0.000195,0.000060,-0.001714
pasif_musteri,-0.032177,-0.029821,-0.002308,-0.000787,-0.001468,-0.038912,-0.001190,-0.000334,0.044839,-0.002094,...,-0.034854,-0.021401,-0.006284,-0.030609,-0.028173,-0.021489,-0.005787,-0.003876,-0.002675,-0.013398
aktiflestirilmis_musteri,0.004046,-0.003425,-0.002183,-0.001905,-0.001388,-0.036803,-0.001126,-0.000316,-0.002094,0.042522,...,-0.014246,-0.014866,-0.004409,-0.015575,0.003580,-0.009274,-0.005124,0.002584,0.001038,-0.008038


In [17]:
D, V = np.linalg.eig(data.cov())
D

array([ 5.39981675e-01,  2.26867088e-01,  1.05252450e-01,  9.91205909e-02,
        8.03810074e-02,  5.36338824e-02,  4.88245480e-02,  4.00679192e-02,
        3.57094111e-02,  2.90244562e-02,  2.19455655e-02,  1.40505441e-02,
        9.50070947e-03,  7.88957597e-03,  7.29354371e-03,  3.06380634e-03,
        2.31828037e-03,  1.58610092e-04,  5.56169015e-04,  6.87632040e-04,
        1.58908982e-03,  1.33731864e-03,  1.24781468e-03, -1.14209975e-16])

In [18]:
for i in range(len(data.columns)):
    c = copy.deepcopy(list(data.columns))
    var = c.pop(i)
    D, V = np.linalg.eig(data[c].cov())
    print("index: {}, eigen_vals: {}\n".format(var, D))

index: tavsiye, eigen_vals: [ 5.06560972e-01  2.25087808e-01  1.05011671e-01  8.38993605e-02
  6.78472629e-02  5.34805870e-02  4.79318540e-02  3.91258881e-02
  3.02200670e-02  2.19503182e-02  1.40647101e-02  9.60824750e-03
  7.92831141e-03  7.43520723e-03  3.06385831e-03  2.31993080e-03
  1.58644601e-04  5.64832547e-04  6.96922433e-04  1.59323959e-03
  1.33805243e-03  1.24820062e-03 -1.10900788e-16]

index: memnuniyet, eigen_vals: [ 4.86827230e-01  2.23920660e-01  1.04889755e-01  9.69045090e-02
  7.87899996e-02  5.21924328e-02  4.81346794e-02  3.83579058e-02
  3.50705529e-02  2.83011222e-02  2.15270792e-02  1.08439043e-02
  7.90625061e-03  7.38730418e-03  3.06689330e-03  2.31873081e-03
  1.58636223e-04  5.57507860e-04  6.89643074e-04  1.59132503e-03
  1.33830099e-03  1.24809533e-03 -1.20456458e-16]

index: yeni_musteri, eigen_vals: [5.33018474e-01 2.26424932e-01 1.05252417e-01 9.83248481e-02
 7.62079748e-02 4.99050105e-02 4.01736111e-02 3.72560671e-02
 2.97438734e-02 2.27335195e-02 1.9

In [19]:
reduced_form, inds = sympy.Matrix(data.cov()).rref()

In [20]:
inds

(0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23)

In [21]:
data.to_csv("../data/generated.csv", sep="\t", index=False)